In [74]:
# import modules
import glob
import pandas as pd
import geopandas as gpd

In [75]:
# read files
filepaths = glob.glob('data/travel_times/*.txt')
grid = gpd.read_file('data/MetropAccess_YKR_grid_EurefFIN.shp')
targets = gpd.read_file('data/shopping_centers/shopping_centers.shp')

# print column names
print('grid cols: ', grid.columns)
print('targets cols: ', targets.columns)

grid cols:  Index(['x', 'y', 'YKR_ID', 'geometry'], dtype='object')
targets cols:  Index(['address', 'id', 'name', 'addr', 'geometry'], dtype='object')


In [76]:
# check if CRS of layers match (-> False)
print('CRS match:', targets.crs == grid.crs)

CRS match: False


In [77]:
# reproject targets to grid CRS
targets = targets.to_crs(grid.crs)
# CRS should now match
print('CRS match:', targets.crs == grid.crs)

CRS match: True


In [78]:
# join ykr grid info to targets
targets_ykr = gpd.sjoin(targets, grid, how="inner", op="within")
# get the ids as list
ykr_ids = targets_ykr.YKR_ID.tolist()
print('YKR ids:', ykr_ids)

YKR ids: [5944003, 5973736, 5978593, 5934891, 5878070, 5963664]


In [79]:
# gather filepaths to selected travel times to list
folder = 'data/HelsinkiRegion_TravelTimeMatrix2015/'
filepaths = []
for id in ykr_ids:
    subfolder = str(id)[:4]+'xxx/'
    filename = 'travel_times_to_ '+str(id)+'.txt'
    filepaths.append(folder + subfolder + filename)

In [80]:
# read files from filepaths
tt_dfs = []
for idx, fp in enumerate(filepaths):
    data = pd.read_csv(fp, sep=';')
    data = data[['from_id', 'pt_r_t']]
    tt_dfs.append(data)

# rename travel time columns in dataframes
t_columns = []
for idx, tt_df in enumerate(tt_dfs):
    columnn = 'pt_r_t_'+str(idx)
    tt_df.columns = ['from_id', columnn]
    t_columns.append(columnn)

# combine travel times
ttimes = grid    
for tt_df in tt_dfs:
    ttimes = pd.merge(ttimes, tt_df, how='inner', left_on='YKR_ID', right_on='from_id')


In [87]:
# select only relevant columns from the data
columns = ['x', 'y', 'YKR_ID', 'geometry']
ttimes = ttimes[columns + t_columns]
ttimes.head(3)

,x,y,YKR_ID,geometry,pt_r_t_0,pt_r_t_1,pt_r_t_2,pt_r_t_3,pt_r_t_4,pt_r_t_5
0,381875.0,6697880.0,5785640,"POLYGON ((382000.0001388059 6697750.000128186,...",132,109,141,109,101,124
1,382125.0,6697880.0,5785641,"POLYGON ((382250.00013875 6697750.000128181, 3...",135,112,143,112,108,127
2,382375.0,6697880.0,5785642,"POLYGON ((382500.0001386951 6697750.000128172,...",137,113,145,114,109,128


In [88]:
# save travel times to file
outfp = 'results/travel_times_raw.shp'
ttimes.to_file(outfp)